This homework is intended for the period between November 30th and January 11th.




# Calculating and Analyzing Climate Indices Using SST Anomalies

In this exercise, you’ll calculate SST anomaly-based indices like the Dipole Mode Index (DMI), Niño 3.4, and the Tropical Northern Atlantic (TNA) index. The TNA is a climate variability index tracking SST anomalies in the eastern tropical North Atlantic, which influence precipitation in South America. These indices are essential for understanding climate patterns. You’ll adjust existing functions to analyze climate events through composites. You'll also include latitude weighting for accurate regional SST representation.

**<p style="color:royalblue;">The effect of latitudinal weighting is minimal in our case, as the spatial regions we average are close to the equator. This method becomes more important when averaging globally or when anomalies within the box vary significantly from grid cell to grid cell. However, for consistency, it is good practice to apply it in all cases.</p>**

## Tasks
The code cell below contains the function you need to modify to include calculations for the Niño 3.4 and TNA indices. Use the DMI calculation as a reference, as it is already implemented in the code. Your task is to extend this function to calculate the Niño 3.4 and TNA indices, incorporating latitude weighting for accuracy.


### 1. Calculate Indices
- **Niño 3.4**: Copy code from [`enso_functions.py`](../Session2_DataHandling/enso_functions.py), add latitude weighting, and verify results.
- **TNA**: Use [NOAA's bounding box for TNA](https://stateoftheocean.osmc.noaa.gov/sur/atl/tna.php). Calculate a 3-month rolling mean and define positive events for June-August with values >0.4°C.
- Reference the DMI calculation to see how to implement latitude weighting.

### 2. Define Positive and Negative Events
- Set threshold-based criteria to define events for each index.

### 3. Generate Composites
- Use provided functions to visualize SST anomaly composites for positive and negative events.

### 4. Analysis
- Discuss how using June-August months for TNA and the rolling mean impact event identification.




In [ ]:
# import necessary library

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import xarray as xr
from scipy.signal import detrend

In [ ]:
# modify functions

def calculate_index(sst_anom_detrended, index_name):
    """
    Generalized function to calculate climate indices based on SST anomalies and specified region(s).
    """

    # Calculate latitude weights
    latitudes = sst_anom_detrended['lat']
    weights = np.cos(np.deg2rad(latitudes)).where(~sst_anom_detrended[0].isnull()).fillna(0)
    ## test without lat weighting - effect negligible for near equator boxes
    #weights = xr.DataArray(1, coords=sst_anom_detrended[0].coords, dims=sst_anom_detrended[0].dims)
    #weights = weights.where(~sst_anom_detrended[0].isnull()).fillna(0)
    


    if index_name.lower() == 'dmi':
        # Define regions for DMI (IOD Index): Western and Eastern Indian Ocean regions
        region1 = sst_anom_detrended.sel(lat=slice(10, -10), lon=slice(50, 70))
        region2 = sst_anom_detrended.sel(lat=slice(0, -10), lon=slice(90, 110))
        # Apply latitude weights and calculate the weighted mean for both regions
        weighted_region1 = (region1 * weights).mean(dim=['lat', 'lon'])
        weighted_region2 = (region2 * weights).mean(dim=['lat', 'lon'])
        index = (weighted_region1 - weighted_region2).rolling(time=3, center=True).mean()

    elif index_name.lower() == 'nino3.4':
        
    # Add additional elif case for tna  here...

    else:
        raise ValueError(f"Index '{index_name}' not recognized.")

    return index

def calculate_composites(anom_detrended, index, index_name):
    """
    Calculate composites for positive and negative events based on the given index and climate pattern.
    """
    if index_name.lower() == 'dmi':
        # Define criteria for positive and negative IOD events (DMI)
        positive_events = ((index > 0.4).astype('b').rolling(time=3, center=True).sum() >= 3) & index['time.month'].isin([6, 7, 8, 9, 10, 11])
        negative_events = ((index < -0.4).astype('b').rolling(time=3, center=True).sum() >= 3) & index['time.month'].isin([6, 7, 8, 9, 10, 11])

    elif index_name.lower() == 'nino3.4':
        # Define criteria for positive and negative ENSO events, use enso_functions.py as a reference

    elif index_name.lower() == 'tna':
        # Define criteria for positive and negative TNA events in JJA (June, July, August)    

    

    else:
        raise ValueError(f"Index '{index_name}' not recognized.")

    # Calculate the average anomaly during positive and negative events
    anom_positive = anom_detrended.where(positive_events).mean(dim='time')
    anom_negative = anom_detrended.where(negative_events).mean(dim='time')

    return anom_positive, anom_negative

def plot_composites(anom_positive, anom_negative, vmin=-1.5, vmax=1.5, label='°C', variable='SST', index_name='Index', 
                    lon_range=None, lat_range=None):
    """
    Plot composites for positive and negative events based on the climate index provided, 
    with an option to focus on a specific region.
    
    Parameters:
    - anom_positive, anom_negative: xarray DataArrays for positive and negative anomalies
    - vmin, vmax: float, value range for color scale
    - label: str, colorbar label
    - variable: str, variable name to display in titles
    - index_name: str, index name to display in titles
    - lon_range: tuple of floats, (min_lon, max_lon) for longitude slicing
    - lat_range: tuple of floats, (min_lat, max_lat) for latitude slicing
    """
    # Slice the data for the specified region, if provided
    if lon_range:
        anom_positive = anom_positive.sel(lon=slice(lon_range[0], lon_range[1]))
        anom_negative = anom_negative.sel(lon=slice(lon_range[0], lon_range[1]))
    if lat_range:
        anom_positive = anom_positive.sel(lat=slice(lat_range[0], lat_range[1]))
        anom_negative = anom_negative.sel(lat=slice(lat_range[0], lat_range[1]))

    # Plotting
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 5), sharey=True)

    anom_positive.plot(ax=axes[0], cbar_kwargs={'label': label}, vmin=vmin, vmax=vmax, cmap='RdBu_r')
    axes[0].set_title(f'{variable} Anomaly - Positive {index_name} Event')

    anom_negative.plot(ax=axes[1], cbar_kwargs={'label': label}, vmin=vmin, vmax=vmax, cmap='RdBu_r')
    axes[1].set_title(f'{variable} Anomaly - Negative {index_name} Event')

    plt.tight_layout()
    plt.show()




Load SST-Data for the period 1960-2023 (again) to analyze composites.

In [ ]:
url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc' 
# or, if data are already stored locally use filename (modify path if necessary) and replace url by filename in open_dataset
# fileneme = '../Data/SST/sst.mnmean.nc'
ds_sst = xr.open_dataset(url).sel(time=slice('1960-01-01', '2023-12-31'))
ds_sst

Calculate the detrended SST anomalies—you probably know this by heart by now.

In [ ]:
# your code here 

Using the functions to calculate the DMI and analyze the IOD composites

In [ ]:
# Step 1: Calculate the  index using the SST anomalies
dmi_index = calculate_index(sst_anom_detrended,'dmi')

# Step 2: Calculate the SST composites for positive and negative  events
sst_anom_positive, sst_anom_negative = calculate_composites(sst_anom_detrended,dmi_index,'dmi')

# Step 3: Plot the SST composites
plot_composites(sst_anom_positive, sst_anom_negative,vmin=-1, vmax=1,lat_range=(20,-20),lon_range=(40,120), index_name='IOD')

Your ENSO Analysis here:

In [ ]:
# your code here 

Your TNA Analysis here:

In [ ]:
# your code here

Your Observations:
....





## Preperation Session 5 and 6:

For a general overview of how object-oriented modeling and programming work, please visit this 
[Storymap](https://storymaps.arcgis.com/stories/dd9d06f89a63400c96927de117a5b28a). The text is in German, but most browsers offer automatic translation. The article provides a closer look at the programming paradigm and dives into the benefits of encapsulation, inheritance, polymorphism, and abstraction.

Please download and replace the updated notebooks for Session 5 and Session 6. Use [update_notebooks.ipynb](../update_notebooks.ipynb) script for this purpose. I will share the `github_urls` and `local_paths` a few days before our next meeting on January 11, 2025, via Ahoi mail.